In [1]:
%matplotlib Tk
import numpy as np
from sympy import Matrix
import matplotlib.pyplot as plt
from IPython.display import clear_output
from matplotlib.animation import FuncAnimation, ArtistAnimation

plt.style.use('seaborn-pastel')

In [2]:
def excite_atoms(positions, states):
    for i in range(len(positions)):
        states[positions[i][0], positions[i][1]] = 1
    return

def random_seeds_displacement(N_atoms_per_row, N_seeds):
    seeds_positions = np.random.choice(range(N_atoms_per_row**2), N_seeds, replace=False)
    x_positions = [seeds_positions[i]//N_atoms_per_row for i in range(N_seeds)]
    y_positions = [seeds_positions[i]%N_atoms_per_row for i in range(N_seeds)]
    return x_positions, y_positions
        
def get_neighbourhood(excited_atom, N_atoms_per_row):
    neighbours = [
                (excited_atom[0]-1, excited_atom[1]), 
                (excited_atom[0]+1, excited_atom[1]),
                (excited_atom[0], excited_atom[1]-1), 
                (excited_atom[0], excited_atom[1]+1)
                ]
    for neighbour in neighbours.copy():
        if -1 in list(neighbour) or N_atoms_per_row in list(neighbour):
            neighbours.remove(neighbour)
    return neighbours

In [3]:
N_steps = 300
N_atoms_per_row = 40
mu_exct_on_resonance = 0.1
p_exct_OFF = 0.1
p_spontaneus_decay = 0.3

N_seeds = np.random.poisson(mu_exct_on_resonance) # extract from a poassonian the inital seeds
x_excited, y_excited = random_seeds_displacement(N_atoms_per_row=N_atoms_per_row, N_seeds=N_seeds) # extract the random positions of the seeds
excited_atoms = list(zip(x_excited, y_excited))

fig = plt.figure(figsize=(13,10))
ims = []

for i in range(N_steps):
    # first phase: check if the neighbours are already excited excite them with a certain probability.
    for excited_atom in excited_atoms.copy():
        neighbours = get_neighbourhood(excited_atom, N_atoms_per_row)
        for neighbour in neighbours:
            if neighbour not in excited_atoms and np.random.random()<p_exct_OFF:
                excited_atoms.append(neighbour)
    # second phase: spontaneous decay of atoms.
    for excited_atom in excited_atoms.copy():
        if np.random.random()<p_spontaneus_decay:
            excited_atoms.remove(excited_atom)
    
    N_seeds = np.random.poisson(mu_exct_on_resonance) # extract from a poassonian the inital seeds
    x_excited, y_excited = random_seeds_displacement(N_atoms_per_row=N_atoms_per_row, N_seeds=N_seeds) # extract the random positions of the seeds

    excited_atoms = excited_atoms + list(zip(x_excited, y_excited))

    if i%2 == 0:
        states = np.zeros([N_atoms_per_row, N_atoms_per_row])
        excite_atoms(excited_atoms, states)

        im = plt.imshow(states, animated=True)
        ims.append([im])

ani = ArtistAnimation(fig, ims, interval=100, blit=True, repeat_delay=10)
ani.save('absorbing_phase_2d.mp4')